In [1]:
# Import fablib library
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

# Initialize fablib
fablib = fablib_manager()

In [2]:
# Create a slice
slice = fablib.new_slice(name="dh-aes-exemplar-extended")

In [3]:
# Define sites
sites = {
    'server': 'LOSA',
    'switch': 'EDC',
    'clients': 'FIU'
}

print(f"The selected sites are {sites['server']}, {sites['switch']}, {sites['clients']}")

# Initialize dictionary to hold all nodes and their details
nodes = {}

# Add server
nodes['server'] = {
    'node': slice.add_node(name="server1", site=sites['server'], cores=4, ram=8, disk=20, image='default_ubuntu_20'),
    'interfaces': []
}

# Add clients
num_clients = 7
for i in range(1, num_clients + 1):
    client_name = f"sta{i}"
    nodes[client_name] = {
        'node': slice.add_node(name=client_name, site=sites['clients'], cores=4, ram=8, disk=20, image='default_ubuntu_20'),
        'interfaces': []
    }

# Add switches
num_switches = 3
for i in range(1, num_switches + 1):
    switch_name = f"switch{i}"
    nodes[switch_name] = {
        'node': slice.add_node(name=switch_name, site=sites['switch'], cores=32, ram=16, disk=40, image='default_ubuntu_20'),
        'interfaces': []
    }

The selected sites are LOSA, EDC, FIU


In [4]:
# Add network interfaces to each node
for name, node_info in nodes.items():
    if name == 'switch1':
        num_interfaces = 8
    elif 'switch' in name:
        num_interfaces = 2
    else:
        num_interfaces = 1
    for j in range(num_interfaces):
        component_name = f"{name}_nic{j+1}"  # Construct the component name
        interface = node_info['node'].add_component(model='NIC_Basic', name=component_name).get_interfaces()[0]
        node_info['interfaces'].append(interface)

In [6]:
# Setup networks
networks = {}

# Connecting clients to the first switch
for i in range(1, 8):
    client_name = f"sta{i}"
    networks[f'net_client_{i}'] = slice.add_l2network(name=f'net_client_{i}', interfaces=[
        nodes[client_name]['interfaces'][0],
        nodes['switch1']['interfaces'][i]
    ])
    
networks['switch-switch1'] = slice.add_l2network(name=f'switch-switch1', interfaces=[
        nodes['switch2']['interfaces'][0],
        nodes['switch1']['interfaces'][0]
    ])

networks['switch-switch2'] = slice.add_l2network(name=f'switch-switch2', interfaces=[
        nodes['switch2']['interfaces'][1],
        nodes['switch3']['interfaces'][0]
    ])

networks['switch-server'] = slice.add_l2network(name=f'switch-server', interfaces=[
        nodes['server']['interfaces'][0],
        nodes['switch3']['interfaces'][1]
    ])

In [7]:
# Submit the slice
slice.submit()


Retry: 18, Time: 1318 sec


ID,2275f832-5ea0-4dff-8a96-19880a6541b1
Name,dh-aes-exemplar-extended
Lease Expiration (UTC),2024-04-13 19:46:22 +0000
Lease Start (UTC),2024-04-12 19:46:23 +0000
Project ID,6ce270de-788d-4e07-8bae-3206860a6387
State,StableOK


ID,Name,Cores,RAM,Disk,Image,Image Type,Host,Site,Username,Management IP,State,Error,SSH Command,Public SSH Key File,Private SSH Key File
7b18b54d-cc18-45eb-9c85-311430e41fde,server1,4,8,100,default_ubuntu_20,qcow2,losa-w2.fabric-testbed.net,LOSA,ubuntu,2001:400:a100:3070:f816:3eff:fe78:982f,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3070:f816:3eff:fe78:982f,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
7eaf0b66-f209-4ee0-b9b4-68a580a14d98,sta1,4,8,100,default_ubuntu_20,qcow2,fiu-w1.fabric-testbed.net,FIU,ubuntu,131.94.57.21,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@131.94.57.21,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
38965169-f8f2-47d2-b84c-f4ca72acf532,sta2,4,8,100,default_ubuntu_20,qcow2,fiu-w1.fabric-testbed.net,FIU,ubuntu,131.94.57.43,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@131.94.57.43,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
d86e7055-38de-4c14-bf55-af37ee9ccdd1,sta3,4,8,100,default_ubuntu_20,qcow2,fiu-w1.fabric-testbed.net,FIU,ubuntu,131.94.57.41,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@131.94.57.41,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
3b8b8d84-7986-4955-b360-6adbafd13436,sta4,4,8,100,default_ubuntu_20,qcow2,fiu-w1.fabric-testbed.net,FIU,ubuntu,131.94.57.28,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@131.94.57.28,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
c46b19ac-c2a8-4270-b643-897a21efb3ec,sta5,4,8,100,default_ubuntu_20,qcow2,fiu-w1.fabric-testbed.net,FIU,ubuntu,131.94.57.49,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@131.94.57.49,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
9c197293-0f6c-46d4-9365-2d287f04fd40,sta6,4,8,100,default_ubuntu_20,qcow2,fiu-w1.fabric-testbed.net,FIU,ubuntu,131.94.57.16,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@131.94.57.16,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
cb6b0924-7f10-45e8-a0ae-72fb3eb5cfae,sta7,4,8,100,default_ubuntu_20,qcow2,fiu-w1.fabric-testbed.net,FIU,ubuntu,131.94.57.13,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@131.94.57.13,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
154f83ed-89e8-46d9-b36d-439df85dbbff,switch1,32,16,100,default_ubuntu_20,qcow2,edc-w1.fabric-testbed.net,EDC,ubuntu,2620:0:c80:1003:f816:3eff:febe:6dda,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2620:0:c80:1003:f816:3eff:febe:6dda,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
bbf12169-efbb-428e-9e25-267e7f455f0e,switch2,32,16,100,default_ubuntu_20,qcow2,edc-w1.fabric-testbed.net,EDC,ubuntu,2620:0:c80:1003:f816:3eff:fec3:cda5,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2620:0:c80:1003:f816:3eff:fec3:cda5,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key


ID,Name,Layer,Type,Site,Subnet,Gateway,State,Error
0087b8da-5bc5-4f4f-9cf5-a4587f03d66b,net_client_1,L2,L2STS,None,None,None,Active,
cddc5402-0063-4335-b434-c05e0f28f80a,net_client_2,L2,L2STS,None,None,None,Active,
1243f138-b5ea-4e9f-9c2e-beaf52c613a0,net_client_3,L2,L2STS,None,None,None,Active,
0764a48e-adfa-4ee1-8c9d-d370fa755934,net_client_4,L2,L2STS,None,None,None,Active,
46072e74-7afe-457e-8a49-99d93c35fe89,net_client_5,L2,L2STS,None,None,None,Active,
113701f5-b90d-4a42-aebd-4c824b75910a,net_client_6,L2,L2STS,None,None,None,Active,
bac8eb7b-b256-45d5-8444-953868fa3a15,net_client_7,L2,L2STS,None,None,None,Active,
8e59792f-f957-403d-a17d-8d2d992dcbf9,switch-server,L2,L2STS,None,None,None,Active,
8098e085-2004-4be2-9a5c-a04d404be13c,switch-switch1,L2,L2Bridge,EDC,None,None,Active,
8f321173-4d95-4171-9f2b-1fdb9b81e0f4,switch-switch2,L2,L2Bridge,EDC,None,None,Active,


Name,Short Name,Node,Network,Bandwidth,Mode,VLAN,MAC,Physical Device,Device,IP Address,Numa Node
server1-server_nic1-p1,p1,server1,switch-server,100,config,,1A:B8:E4:DB:34:62,enp6s0,enp6s0,None,4
sta1-sta1_nic1-p1,p1,sta1,net_client_1,100,config,,2A:94:B1:9B:D0:7C,enp6s0,enp6s0,None,6
sta2-sta2_nic1-p1,p1,sta2,net_client_2,100,config,,2A:98:1B:49:02:A6,enp6s0,enp6s0,None,6
sta3-sta3_nic1-p1,p1,sta3,net_client_3,100,config,,2A:C2:85:86:8C:DD,enp6s0,enp6s0,None,6
sta4-sta4_nic1-p1,p1,sta4,net_client_4,100,config,,2E:33:03:3B:DC:D8,enp6s0,enp6s0,None,6
sta5-sta5_nic1-p1,p1,sta5,net_client_5,100,config,,2E:74:33:C5:52:E0,enp6s0,enp6s0,None,6
sta6-sta6_nic1-p1,p1,sta6,net_client_6,100,config,,2E:BD:C3:6F:A1:78,enp6s0,enp6s0,None,6
sta7-sta7_nic1-p1,p1,sta7,net_client_7,100,config,,32:35:83:40:01:4F,enp6s0,enp6s0,None,6
switch1-switch1_nic1-p1,p1,switch1,switch-switch1,100,config,,1A:DD:17:77:93:A5,enp8s0,enp8s0,None,6
switch1-switch1_nic2-p1,p1,switch1,net_client_1,100,config,,26:64:7F:1C:74:24,enp11s0,enp11s0,None,6



Time to print interfaces 1349 seconds


'2275f832-5ea0-4dff-8a96-19880a6541b1'

In [8]:
import pandas as pd
pd.set_option('display.max_colwidth', None)
slice_info = [{'Name': n.get_name(), 'SSH command': n.get_ssh_command()} for n in slice.get_nodes()]
pd.DataFrame(slice_info).set_index('Name')

,SSH command
Name,
server1,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3070:f816:3eff:fe78:982f
sta1,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@131.94.57.21
sta2,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@131.94.57.43
sta3,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@131.94.57.41
sta4,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@131.94.57.28
sta5,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@131.94.57.49
sta6,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@131.94.57.16
sta7,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@131.94.57.13
switch1,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2620:0:c80:1003:f816:3eff:febe:6dda
